# ChromaDB quickstart

ChromaDB uses the Hierarchical navigable small world (HNSW) approximate nearest neighbour search to search for similar embeddings.

In [1]:
import chromadb
from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction

ollama_ef = OllamaEmbeddingFunction(
    url="http://localhost:11434",
    model_name="nomic-embed-text:latest",
)

client = client = chromadb.EphemeralClient() 
# to save the DB on disc, use: chromadb.PersistentClient()
# you can specify the path there, see https://docs.trychroma.com/docs/run-chroma/persistent-client

By default, ChromaDB uses the `all-MiniLM-L6-v2` embedding model (and downloads it automatically) to embed text in a collection. If you want anything else you need to specify it when you create the collection. Here we use an embedding provided through Ollama. We also specify at creation that we want to use the cosine distance (default is L2, the squared Euclidian metric). See documentation: https://docs.trychroma.com/docs/collections/configure

In [2]:
collection = client.create_collection(
    name="CatFacts", 
    embedding_function=ollama_ef,
    metadata={"hnsw:space": "cosine"}  # or "dotproduct"
)

If using `PersistenClient` you can load existing collections, or delete them, or do a load-else-create:

        collection = client.get_collection(name="test") 
        # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
        collection = client.get_or_create_collection(name="test") 
        # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
        client.delete_collection(name="my_collection") 
        # Delete a collection and all associated embeddings, documents, and metadata. 


## Storing our cat facts

In [3]:
import requests

cat_facts_url = 'https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt'
response = requests.get(cat_facts_url)
cat_facts = response.text.split("\n")

In [4]:
%%time
collection.add(
    documents = cat_facts,
    ids = [f"id{iii}" for iii in range(len(cat_facts))],
)

CPU times: user 73.3 ms, sys: 15.3 ms, total: 88.6 ms
Wall time: 2.12 s


`ids` are required. If this id already exists, then the document is not added to the data base.

You can also ad metadata, as a list of dictionaries.

Instead of `documents = ...` you can also pass a list of `embeddings = ...` directly.

https://docs.trychroma.com/docs/collections/add-data

If needed, collections can be renamed: `collection.modify(name="new_name")`

## Accessing the stored data

For each collection an index is created using the hnswlib python library (an implementaion of HNSW approximate nearest neighbor search algorithm)

https://docs.trychroma.com/docs/querying-collections/query-and-get

In [5]:
print(collection.get().keys())
print(len(collection.get()['documents']))
print(collection.count())

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])
150
150


In [6]:
collection.get(
	ids=["id1", "id2", "id3"],
)

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': None,
 'documents': ['Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.',
  'When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.',
  'The technical term for a cat’s hairball is a “bezoar.”'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None, None]}

In [7]:
# peek at the N first items:
collection.peek(2) 

{'ids': ['id0', 'id1'],
 'embeddings': array([[ 0.0231142 ,  0.06630387, -0.17538089, ..., -0.0107691 ,
          0.00104107, -0.01289808],
        [ 0.07731966,  0.07048559, -0.14541987, ..., -0.01019575,
         -0.04358175, -0.05696372]], shape=(2, 768)),
 'documents': ['On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.',
  'Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.'],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': [None, None]}

If your collection has metadata, you can also use e.g. `where={"style": "style1"}` as an argument to the `get` method.

In [8]:
%%time
collection.query(
    query_texts=["Can cats catch illnesses?"],
    n_results=5,
)

CPU times: user 1.83 ms, sys: 614 μs, total: 2.45 ms
Wall time: 12.9 ms


{'ids': [['id131', 'id127', 'id138', 'id126', 'id125']],
 'embeddings': None,
 'documents': [['A form of AIDS exists in cats.',
   'Though rare, cats can contract canine heart worms.',
   'Many people fear catching a protozoan disease, Toxoplasmosis, from cats. This disease can cause illness in the human, but more seriously, can cause birth defects in the unborn. Toxoplasmosis is a common disease, sometimes spread through the feces of cats. It is caused most often from eating raw or rare beef. Pregnant women and people with a depressed immune system should not touch the cat litter box. Other than that, there is no reason that these people have to avoid cats.',
   'Cats can get tapeworms from eating mice. If your cat catches a mouse it is best to take the prize away from it.',
   'Cats can get tapeworms from eating fleas. These worms live inside the cat forever, or until they are removed with medication. They reproduce by shedding a link from the end of their long bodies. This link craw

The query method also allows to filter on kewords, and on metadata:

    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"},